In [2]:
import pandas as pd 
import json
import pymongo
from sklearn.compose import ColumnTransformer
import numpy as np

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['restuarant']
mycollection = db['me']

In [4]:
all_records = mycollection.find()

In [5]:
list_cursor  = list(all_records)

In [6]:
df = pd.DataFrame(list_cursor)

In [7]:
df.head()

,_id,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,6215b143754b34e60574f6e1,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,6215b143754b34e60574f6e2,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,6215b143754b34e60574f6e3,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,6215b143754b34e60574f6e4,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,6215b143754b34e60574f6e5,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [8]:
df.shape

(51717, 18)

In [9]:
df.dtypes

_id                            object
url                            object
address                        object
name                           object
online_order                   object
book_table                     object
rate                           object
votes                          object
phone                          object
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
dtype: object

In [10]:
df.isna().sum()

_id                                0
url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [11]:
df = df.drop(['_id','url','phone','_id','address'],axis=1)
df.drop_duplicates(inplace=True)

In [12]:
df.dropna(how='any',inplace=True)

In [13]:
df.shape

(23395, 14)

In [14]:
df = df.rename(columns={'approx_cost(for two people)' : 'cost' , 'listed_in(type)' : 'type', 'listed_in(city)' : 'city'})

In [15]:
df['cost'] = df['cost'].apply(lambda x : x.replace(',',''))
df['cost'] = df['cost'].astype(float)

In [16]:
df = df[df['rate'] != 'NEW']

In [17]:
df['rate'] = df['rate'].apply(lambda x : x.replace('/5',''))
df['rate'] = df['rate'].astype(float)

In [18]:
df.head()

,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,type,city
0,Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [20]:
df.online_order[df.online_order == "Yes"] = 1 
df.online_order[df.online_order == "No"] = 0
df.book_table[df.book_table == "Yes"] = 1
df.book_table[df.book_table == "No"] = 0


C:\Users\VAISHNO RAIZADA\AppData\Local\Temp\ipykernel_7864\2331746515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.online_order[df.online_order == "Yes"] = 1
C:\Users\VAISHNO RAIZADA\AppData\Local\Temp\ipykernel_7864\2331746515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.online_order[df.online_order == "No"] = 0
C:\Users\VAISHNO RAIZADA\AppData\Local\Temp\ipykernel_7864\2331746515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [21]:
x = df.iloc[:,[1,2,4,5,6,8,9]]
x.head()

,online_order,book_table,votes,location,rest_type,cuisines,cost
0,1,1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0
1,1,0,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0
2,1,0,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0
3,0,0,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0
4,0,0,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0


In [22]:
y = df['rate']
y.head()

0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64

In [23]:
df.isna().sum()

name            0
online_order    0
book_table      0
rate            0
votes           0
location        0
rest_type       0
dish_liked      0
cuisines        0
cost            0
reviews_list    0
menu_item       0
type            0
city            0
dtype: int64

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.pipeline import  Pipeline

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [26]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression

In [27]:
lr = LinearRegression()
dt = DecisionTreeRegressor(max_depth=5)
rf = RandomForestRegressor(n_estimators=50, random_state=2)
gb = GradientBoostingRegressor(n_estimators=50, random_state=2)
et = ExtraTreesRegressor(n_estimators=50, random_state=2)
bag = BaggingRegressor(n_estimators=50, random_state=2)
knn = KNeighborsRegressor()
ab = AdaBoostRegressor(n_estimators=50, random_state=2)

In [28]:
clfs1 = {    'LR'  :  lr    ,
             'DT' :  dt ,
             'RF' :  rf ,
             'GB' :  gb ,
             'ETR' :  et ,
             'BAGR' :  bag,
             'KNN' :  knn,
             'ABR' :  ab,
}

In [30]:
r2score = []
mae_score = []
mse_score = []
rmse_score = []
adj_r_squared_score_score = []


for name,clf in clfs1.items():
    categorical_features = ["location", "rest_type", "cuisines"]

    categorical_transformer = OneHotEncoder(sparse=False,handle_unknown="ignore")

    step1 =  ColumnTransformer([('ohe',categorical_transformer,categorical_features)], remainder='passthrough')

    step2 = clf

    pipe = Pipeline([('step1',step1),('clf',clf)])

    pipe.fit(x_train,y_train)

    y_pred = pipe.predict(x_test)
    
    r2score.append(r2_score(y_test,y_pred))
    mae_score.append(mean_absolute_error(y_test,y_pred))
    mse_score.append(mean_squared_error(y_test,y_pred))
    rmse_score.append(np.sqrt(mean_squared_error(y_test,y_pred)))
    adj_r_squared_score_score.append(1 - ((1 - (r2_score(y_test,y_pred)) )*(6975 - 1)/ (6975 - 1 - 9)))

In [31]:
performance_df = pd.DataFrame({'Algorithm':clfs1.keys(), 'R2_Score' :r2score, 'Adjusted R2_Score' : adj_r_squared_score_score, 'Root Mean Square Error' :rmse_score ,'Mean Absolute Error' : mae_score, 'Mean Squared Error' : mse_score}).sort_values('R2_Score',ascending=False)

In [32]:
performance_df

,Algorithm,R2_Score,Adjusted R2_Score,Root Mean Square Error,Mean Absolute Error,Mean Squared Error
4,ETR,9.418895e-01,9.418144e-01,0.102978,0.024377,1.060455e-02
5,BAGR,9.082151e-01,9.080965e-01,0.129421,0.047911,1.674978e-02
2,RF,9.080825e-01,9.079637e-01,0.129514,0.048092,1.677398e-02
6,KNN,6.189019e-01,6.184095e-01,0.263716,0.143808,6.954638e-02
3,GB,3.958136e-01,3.950328e-01,0.332051,0.234992,1.102577e-01
1,DT,3.140197e-01,3.131333e-01,0.353814,0.251305,1.251842e-01
7,ABR,-1.185033e-02,-1.315781e-02,0.429712,0.360010,1.846520e-01
0,LR,-6.189469e+11,-6.197467e+11,336082.265805,13201.408259,1.129513e+11


In [ ]:
import pickle

In [ ]:
# pickle.dump(pipe,open('ExtraTreesRegressor.pkl','wb'))

In [ ]:
# pipe.predict(pd.DataFrame([['Yes','Yes',400, 'Jayanagar', 'Pub', 'BBQ',1200 ]],columns=['online_order','book_table','votes','location', 'rest_type','cuisines','cost']))